## daTAanalysis.R
### Jay Sayre - sayrejay (at) gm@i|,

NOTE: cleanuptextable.py MUST BE RUN AFTER RUNNING THIS SCRIPT TO UPDATE TABLES IN PAPER

## INPUTS: 
"municipality_level_DATASET.csv" - As titled, produced by dataassembly.py

"mun_level_isic4dig_DATASET.csv" - Same as above, except uses ISIC 4 digit and only empresas data (not IPUMS), produced by dataassembly.py

"municipality_occupation_level_DATASET.csv" - As titled, produced by dataassembly.py

'municipality_migration_DATASET.csv' - contains contains population estimates for 2002 and 2010 and tariff levels for 2002 and 2010 at the municipality level

## OUTPUTS:

"../Plots/munregs.tex" - Municipality level results

"../Plots/mun4digregs.tex" - Municipality level results (ISIC 4 digit/no IPUMS occupation)

"../Plots/munoccregs.tex" - Occupation level results

"../Plots/migregs.tex" - Municipality level results for migration info

In [1]:
library(ggplot2)
library(texreg)
library(tikzDevice)

## Directory information
setwd("~/Dropbox/College/DR_Paper/cafta-dr/Output/")
#setwd("D:/Dropbox/Dropbox (Personal)/College/DR_Paper/")
plotdir <- "../Plots/"

## INPUTS
mundf <- read.csv("municipality_level_DATASET.csv")
munisic4df <- read.csv("mun_level_isic4dig_DATASET.csv")
munoccdf <- read.csv("municipality_occupation_level_DATASET.csv")
munmigdf <- read.csv("municipality_migration_DATASET.csv")

## OUTPUTS 
munlevelresults <- paste(plotdir,"munregs.tex",sep="")
mun4diglevelresults <- paste(plotdir,"mun4digregs.tex",sep="")
occupationlevelresults <- paste(plotdir,"munoccregs.tex",sep="")
migresults <- paste(plotdir,"migregs.tex",sep="")


Version:  1.36
Date:     2015-12-08
Author:   Philip Leifeld (Eawag & University of Bern)

Please cite the JSS article in your publications -- see citation("texreg").


In [2]:
### For clustered standard errors
### http://www.r-bloggers.com/texreg-a-package-for-beautiful-and-easily-customizable-latex-regression-tables-from-r/
                    
robust.se <- function(model, cluster){
  require(sandwich)
  M <- length(unique(cluster))
  N <- length(cluster)
  K <- model$rank
  dfc <- (M/(M - 1)) * ((N - 1)/(N - K))
  uj <- apply(estfun(model), 2, function(x) tapply(x, cluster, sum));
  rcse.cov <- dfc * sandwich(model, meat = crossprod(uj)/N)
  rcse.se <- coeftest(model, rcse.cov)
  return(list(rcse.cov, rcse.se))
}
                        
coeftest <- function(x, vcov. = NULL, df = NULL, ...)
{
  UseMethod("coeftest")
}

coeftest.default <- function(x, vcov. = NULL, df = NULL, ...)
{
  ## use S4 methods if loaded
  coef0 <- if("stats4" %in% loadedNamespaces()) stats4::coef else coef
  vcov0 <- if("stats4" %in% loadedNamespaces()) stats4::vcov else vcov

  ## extract coefficients and standard errors
  est <- coef0(x)
  if(is.null(vcov.)) se <- vcov0(x) else {
      if(is.function(vcov.)) se <- vcov.(x)
        else se <- vcov.
  }
  se <- sqrt(diag(se))

  ## match using names and compute t/z statistics
  if(!is.null(names(est)) && !is.null(names(se))) {
    if(length(unique(names(est))) == length(names(est)) && length(unique(names(se))) == length(names(se))) {
      anames <- names(est)[names(est) %in% names(se)]
      est <- est[anames]
      se <- se[anames]
    }
  }  
  tval <- as.vector(est)/se

  ## apply central limit theorem
  if(is.null(df)) {
    df <- try(df.residual(x), silent = TRUE)
    if(inherits(df, "try-error")) df <- NULL
  }
  if(is.null(df)) df <- 0

  if(is.finite(df) && df > 0) {
    pval <- 2 * pt(abs(tval), df = df, lower.tail = FALSE)
    cnames <- c("Estimate", "Std. Error", "t value", "Pr(>|t|)")
    mthd <- "t"
  } else {
    pval <- 2 * pnorm(abs(tval), lower.tail = FALSE)
    cnames <- c("Estimate", "Std. Error", "z value", "Pr(>|z|)")
    mthd <- "z"
  }
  rval <- cbind(est, se, tval, pval)
  colnames(rval) <- cnames
  class(rval) <- "coeftest"
  attr(rval, "method") <- paste(mthd, "test of coefficients")
  ##  dQuote(class(x)[1]), "object", sQuote(deparse(substitute(x))))
  return(rval)
} 

coeftest.glm <- function(x, vcov. = NULL, df = Inf, ...)
  coeftest.default(x, vcov. = vcov., df = df, ...)  

coeftest.mlm <- function(x, vcov. = NULL, df = NULL, ...)
{
  ## obtain vcov
  v <- if(is.null(vcov.)) vcov(x) else if(is.function(vcov.)) vcov.(x) else vcov.

  ## nasty hack: replace coefficients so that their names match the vcov() method
  x$coefficients <- structure(as.vector(x$coefficients), .Names = colnames(vcov(x)))

  ## call default method
  coeftest.default(x, vcov. = v, df = df, ...)
}

coeftest.survreg <- function(x, vcov. = NULL, df = Inf, ...)
{
  if(is.null(vcov.)) v <- vcov(x) else {
      if(is.function(vcov.)) v <- vcov.(x)
  	else v <- vcov.
  }
  if(length(x$coefficients) < NROW(x$var)) {
    x$coefficients <- c(x$coefficients, "Log(scale)" = log(x$scale))
  }
  coeftest.default(x, vcov. = v, df = df, ...)  
} 

coeftest.breakpointsfull <- function(x, vcov. = NULL, df = NULL, ...)
{
  est <- coef(x, ...)
  if(is.null(df)) {
    df <- df.residual(x, ...)
    df <- as.vector(rep(df, rep(NCOL(est), length(df))))
  }  

  rnames <- as.vector(t(outer(rownames(est), colnames(est), paste)))
  est <- as.vector(t(est))
  
  se <- vcov(x, vcov. = vcov., ...)

  se <- as.vector(sapply(seq(along = se), function(x) sqrt(diag(se[[x]]))))
  tval <- est/se

  if(any(is.finite(df) && df > 0)) {
    pval <- 2 * pt(abs(tval), df = df, lower.tail = FALSE)
    cnames <- c("Estimate", "Std. Error", "t value", "Pr(>|t|)")
    mthd <- "t"
  } else {
    pval <- 2 * pnorm(abs(tval), lower.tail = FALSE)
    cnames <- c("Estimate", "Std. Error", "z value", "Pr(>|z|)")
    mthd <- "z"
  }
  rval <- cbind(est, se, tval, pval)
  colnames(rval) <- cnames
  rownames(rval) <- rnames
  class(rval) <- "coeftest"
  attr(rval, "method") <- paste(mthd, "test of coefficients")
  ##  dQuote(class(x)[1]), "object", sQuote(deparse(substitute(x))))
  return(rval)
} 

print.coeftest <- function(x, ...)
{
  mthd <- attr(x, "method")
  if(is.null(mthd)) mthd <- "Test of coefficients"
  cat(paste("\n", mthd,":\n\n", sep = ""))
  printCoefmat(x, ...)
  cat("\n")
  invisible(x)
}

In [3]:
### Prepare variables in municipality level dataset
mundf$chnginc1 <- log(mundf$grossalary13)-log(mundf$inc02)
mundf$chnginc2 <- mundf$frstsourcinc13-mundf$inc02
mundf$chnginc3 <- mundf$occinc13-mundf$inc02

mundf$prov <- factor(mundf$prov)
mundf$chngtrf <- log(mundf$duty13+1)-log(mundf$duty02+1)

### Prepare variables in municipality level (ISIC 4) dataset
munisic4df$chnginc1 <- munisic4df$occinc13-munisic4df$inc02
munisic4df$chnginc2 <- log(1+munisic4df$grossalary13)-log(1+munisic4df$inc02)
munisic4df$chnginc3 <- munisic4df$frstsourcinc13-munisic4df$inc02
munisic4df$prov <- factor(munisic4df$prov)
munisic4df$chngtrf <- munisic4df$duty13-munisic4df$duty02

### Prepare variables in municipality/occupation level dataset
munoccdf$chnginc1 <- munoccdf$frstsourcinc13-munoccdf$inc02
munoccdf$chnginc2 <- log(munoccdf$grossalary13)-log(munoccdf$inc02)
munoccdf$chnginc3 <- munoccdf$occinc13-munoccdf$inc02
munoccdf$chngtrf <- log(munoccdf$duty13+1)-log(munoccdf$duty02+1)

munoccdf$chngwrk <- munoccdf$numworkers10-munoccdf$numworkers02
munoccdf$occ <- factor(munoccdf$occ)
munoccdf$munocc <- factor(munoccdf$munocc)

## Subset down to observations without missing variables
munoccdf <- munoccdf[!is.na(munoccdf$chnginc2),]
munoccdf <- munoccdf[!is.nan(munoccdf$chnginc2),]
munoccdf <- munoccdf[!is.infinite(munoccdf$chnginc2),]
munoccdf <- munoccdf[!is.na(munoccdf$chngtrf),]
munoccdf <- munoccdf[!is.nan(munoccdf$chngtrf),]
munoccdf <- munoccdf[!is.infinite(munoccdf$chngtrf),]
munoccdf <- munoccdf[!is.na(munoccdf$munocc), ]
NROW(munoccdf)

## Make a different dataset with only occupations producing tradables
tradedsectordf <- subset(munoccdf, nontraded == 0) 
tradedsectordf <- tradedsectordf[!is.na(tradedsectordf$firmconc02),]
tradedsectordf <- tradedsectordf[!is.na(tradedsectordf$firmconc10),]

NROW(tradedsectordf)
#NROW(subset(munoccdf, nontraded == 1))

### Prepare variables in migration dataset
munmigdf$chngtrf <- munmigdf$duty02-munmigdf$duty10
munmigdf$chngpop <- munmigdf$pop10-munmigdf$pop02
munmigdf$chngempop <- munmigdf$empop02-munmigdf$empop10
munmigdf$mun <- factor(munmigdf$mun)
munmigdf$prov <- factor(munmigdf$prov)

[1] 1184

[1] 978

[1] 204

In [4]:
### Municipality level regressions
#colnames(mundf)
munreg1 <- lm(chnginc1~chngtrf, data = mundf)
munreg2 <- lm(chnginc1~chngtrf+prov, data = mundf)
robmunreg1 <- robust.se(munreg1, mundf$prov)[[2]]
robmunreg2 <- robust.se(munreg2, mundf$prov)[[2]]

#summary(munreg1)
#plot(mundf$chnginc3,mundf$chngtrf)

texreg(list(munreg1,munreg2),#,munreg3),
        file=munlevelresults,
        stars = c(0.01, 0.05,0.10),
        caption="Municipality level regressions",
        caption.above = TRUE, 
        override.se=list(robmunreg1[,2],robmunreg2[,2]),
        override.pval=list(robmunreg1[,4],robmunreg2[,4]),
        omit.coef="prov",
        custom.model.names=c("(1)","(2)"))#,"(3)"))


Loading required package: sandwich
The table was written to the file '../Plots/munregs.tex'.



In [5]:
### Municipality level (ISIC 4 digit) regressions
#colnames(mundf)
mun4reg1 <- lm(chnginc2~chngtrf, data = munisic4df)
mun4reg2 <- lm(chnginc2~chngtrf+prov, data = munisic4df)
mun4reg3 <- lm(chnginc3~chngtrf+prov, data = munisic4df)
robmun4reg1 <- robust.se(mun4reg1, munisic4df$prov)[[2]]
robmun4reg2 <- robust.se(mun4reg2, munisic4df$prov)[[2]]
robmun4reg3 <- robust.se(mun4reg3, munisic4df$prov)[[2]]

#summary(mun4reg1)

texreg(list(mun4reg1,mun4reg2,mun4reg3),
        file=mun4diglevelresults,
        stars = c(0.01, 0.05,0.10),
        caption="Municipality level regressions",
        caption.above = TRUE, 
        override.se=list(robmun4reg1[,2],robmun4reg2[,2],robmun4reg3[,2]),
        override.pval=list(robmun4reg1[,4],robmun4reg2[,4],robmun4reg3[,4]),
        omit.coef="prov",
        custom.model.names=c("(1)","(2)","(3)"))

The table was written to the file '../Plots/mun4digregs.tex'.



In [6]:
### Municipality/occupation level regressions
colnames(munoccdf)

reg1 <- lm(chnginc2~chngtrf,data = tradedsectordf)
reg2 <- lm(chnginc2~chngtrf,data = munoccdf)
reg3 <- lm(chnginc2~chngtrf+munocc,data = tradedsectordf)
reg4 <- lm(chnginc2~chngtrf+munocc,data = munoccdf)
reg5 <- lm(chnginc2~chngtrf+munocc+nontraded,data = munoccdf)
reg6 <- lm(chnginc2~chngtrf+firmconc02+munocc,data = tradedsectordf)
reg7 <- lm(chnginc2~chngtrf+firmconc02*chngtrf+munocc,data = tradedsectordf)
reg8 <- lm(chnginc2~chngtrf+firmconc10+munocc,data = tradedsectordf)
reg9 <- lm(chnginc2~chngtrf+firmconc10*chngtrf+munocc,data = tradedsectordf)
robreg1 <- robust.se(reg1, tradedsectordf$munocc)[[2]]
robreg2 <- robust.se(reg2, munoccdf$munocc)[[2]]
robreg3 <- robust.se(reg3, tradedsectordf$munocc)[[2]]
robreg4 <- robust.se(reg4, munoccdf$munocc)[[2]]
robreg5 <- robust.se(reg5, munoccdf$munocc)[[2]]
robreg6 <- robust.se(reg6, tradedsectordf$munocc)[[2]]
robreg7 <- robust.se(reg7, tradedsectordf$munocc)[[2]]
robreg8 <- robust.se(reg8, tradedsectordf$munocc)[[2]]
robreg9 <- robust.se(reg9, tradedsectordf$munocc)[[2]]

#summary(reg1)
#plot(munoccdf$chnginc2,munoccdf$chngtrf)

texreg(list(reg1,reg2,reg3,reg4,reg5,reg6,reg7),#,reg8,reg9),
        file=occupationlevelresults,
        stars = c(0.01, 0.05,0.10),
        caption="Municipality/occupation level regressions",
        caption.above = TRUE,
        override.se=list(robreg1[,2],robreg2[,2],robreg3[,2],
                        robreg4[,2],robreg5[,2],robreg6[,2],
                        robreg7[,2]),#,robreg8[,2],robreg9[,2]),
        override.pval=list(robreg1[,4],robreg2[,4],robreg3[,4],
                          robreg4[,4],robreg5[,4],robreg6[,4],
                          robreg7[,4]),#robreg8[,4],robreg9[,4]),
        omit.coef="munocc|z",
        custom.model.names=c("(1)","(2)","(3)","(4)","(5)","(6)",
                             "(7)"))#,"(8)","(9)"))


[1] "inc02"          "edu02"          "edu10"          "munocc"        
 [5] "grossalary13"   "occinc13"       "frstsourcinc13" "edu13"         
 [9] "numworkers02"   "numworkers10"   "firmconc02"     "firmconc10"    
[13] "occ"            "duty02"         "duty10"         "duty13"        
[17] "nontraded"      "chnginc1"       "chnginc2"       "chnginc3"      
[21] "chngtrf"        "chngwrk"

The table was written to the file '../Plots/munoccregs.tex'.



In [8]:
summary(reg7)


Call:
lm(formula = chnginc2 ~ chngtrf + firmconc02 * chngtrf + munocc, 
    data = tradedsectordf)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.2626 -0.5246 -0.0227  0.5324  3.1546 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         0.207044   0.272186   0.761 0.447065    
chngtrf             0.412053   0.108035   3.814 0.000147 ***
firmconc02          1.177944   1.245316   0.946 0.344470    
munocc201           0.018891   0.324040   0.058 0.953526    
munocc202           1.464440   0.461573   3.173 0.001565 ** 
munocc203           0.360085   0.461987   0.779 0.435946    
munocc204           0.215277   0.503925   0.427 0.669343    
munocc205          -0.151895   0.462493  -0.328 0.742671    
munocc206           0.270882   0.408363   0.663 0.507297    
munocc208          -0.180758   0.431641  -0.419 0.675491    
munocc209          -0.356802   0.676035  -0.528 0.597786    
munocc301           0.669694   0.391638   1.710 0.087636 . 

In [7]:
### Municipality migration regressions
#colnames(munmigdf)

reg1 <- lm(chngpop~chngtrf,data = munmigdf)
reg2 <- lm(chngpop~chngtrf+prov,data = munmigdf)
reg3 <- lm(chngempop~chngtrf,data = munmigdf)
reg4 <- lm(chngempop~chngtrf+prov,data = munmigdf)
robreg1 <- robust.se(reg1, munmigdf$prov)[[2]]
robreg2 <- robust.se(reg2, munmigdf$prov)[[2]]
robreg3 <- robust.se(reg3, munmigdf$prov)[[2]]
robreg4 <- robust.se(reg4, munmigdf$prov)[[2]]

texreg(list(reg1,reg2,reg3,reg4),
        file=migresults,
        stars = c(0.01, 0.05,0.10),
        caption="Municipality level migration/workforce regressions",
        caption.above = TRUE,
        override.se=list(robreg1[,2],robreg2[,2],robreg3[,2],
                        robreg4[,2]),
        override.pval=list(robreg1[,4],robreg2[,4],robreg3[,4],
                        robreg4[,4]),
        omit.coef="prov|z",
        custom.model.names=c("(1)","(2)","(3)","(4)"))

The table was written to the file '../Plots/migregs.tex'.

